In [1]:
import argparse
import copy
import datetime
import logging
import os
import random
import sys
from random import shuffle
import math
import networkx as nx
import numpy as np
import tensorflow as tf
from tensorflow.python import debug as tf_debug
import sys
sys.path.append("..") 
from deeplp.models.deeplp_att import DeepLP_ATT
from deeplp.models.deeplp_edge import DeepLP_Edge
from deeplp.models.deeplp_wrbf import DeepLP_WRBF
from deeplp.models.lp import LP
from deeplp.utils import (calc_masks, create_seed_features, load_data, load_and_prepare_planetoid_data, create_seed_features_planetoid,
                          num_layers_dict, prepare_data, random_unlabel)

/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
for setting in ['planetoid_balanced', 'planetoid_random', 'planetoid_random_val_as_train']:
    for num_iter in [100,1000]:
        test_accs = np.zeros((3,11))
        unlabeled_accs = np.zeros((3,11))
        gcc_accs = np.zeros((3,11))
        nogcc_accs = np.zeros((3,11))
        for i,data in enumerate(['linqs_cora_planetoid','linqs_citeseer_planetoid','linqs_pubmed_planetoid']):
            for j,seed in enumerate([-1,0,1,2,3,4,5,6,7,8,9]):
                print('---------',setting,num_iter,data,seed,'---------')
                true_labels, features, raw_features, graph, labeled_indices, unlabeled_indices, target_indices, gcc_indices, nogcc_indices, validation_indices = load_and_prepare_planetoid_data(data, setting, seed=seed)
                num_nodes, num_classes = true_labels.shape
                labels, is_labeled = calc_masks(true_labels, labeled_indices,
                                                unlabeled_indices, raw_features, logistic=True)

                lp = LP()
                pred = lp.iter_sp(labels,
                                        graph,
                                        is_labeled,
                                        num_iter,
                                        target_indices)
                y_pred = np.argmax(pred,axis=1)
                y_true = np.argmax(true_labels[target_indices],axis=1)
                acc = np.mean(y_pred == y_true)  
                test_accs[i,j] = acc
                print(acc)

                pred = lp.iter_sp(labels,
                                        graph,
                                        is_labeled,
                                        num_iter,
                                        unlabeled_indices)
                y_pred = np.argmax(pred,axis=1)
                y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
                acc = np.mean(y_pred == y_true)   
                unlabeled_accs[i,j] = acc
                print(acc)

                pred = lp.iter_sp(labels,
                                        graph,
                                        is_labeled,
                                        num_iter,
                                        gcc_indices)
                y_pred = np.argmax(pred,axis=1)
                y_true = np.argmax(true_labels[gcc_indices],axis=1)
                acc = np.mean(y_pred == y_true)  
                gcc_accs[i,j] = acc
                print(acc)

                pred = lp.iter_sp(labels,
                                        graph,
                                        is_labeled,
                                        num_iter,
                                        nogcc_indices)
                y_pred = np.argmax(pred,axis=1)
                y_true = np.argmax(true_labels[nogcc_indices],axis=1)
                acc = np.mean(y_pred == y_true)  
                nogcc_accs[i,j] = acc
                print(acc)
        np.savetxt(f'lp_planetoid-test-{setting}-{num_iter}.csv',test_accs,delimiter=',')
        np.savetxt(f'lp_planetoid-unlabeled-{setting}-{num_iter}.csv',unlabeled_accs,delimiter=',')
        np.savetxt(f'lp_planetoid-gcc-{setting}-{num_iter}.csv',gcc_accs,delimiter=',')
        np.savetxt(f'lp_planetoid-nogcc-{setting}-{num_iter}.csv',nogcc_accs,delimiter=',')

--------- planetoid_balanced 100 linqs_cora_planetoid -1 ---------
========================= 140
fixed seed
========================= 140
Logistic Regression
0.744
0.7324766355140186
0.7465086754126111
0.5707317073170731
--------- planetoid_balanced 100 linqs_cora_planetoid 0 ---------
========================= 140
randomize seed
balanced 20
========================= 140
Logistic Regression
0.7
0.6943146417445483
0.7014861995753715
0.6150234741784038
--------- planetoid_balanced 100 linqs_cora_planetoid 1 ---------
========================= 140
randomize seed
balanced 20
========================= 140
Logistic Regression
0.752
0.757398753894081
0.771780705482363
0.6
--------- planetoid_balanced 100 linqs_cora_planetoid 2 ---------
========================= 140
randomize seed
balanced 20
========================= 140
Logistic Regression
0.75
0.7601246105919003
0.7646559048428208
0.7102803738317757
--------- planetoid_balanced 100 linqs_cora_planetoid 3 ---------
=========================

/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


========================= 60
randomize seed
balanced 20
========================= 60
Logistic Regression
0.724
0.7175052144274304
0.7175052144274304


/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan
--------- planetoid_balanced 100 linqs_pubmed_planetoid 1 ---------
========================= 60
randomize seed
balanced 20
========================= 60
Logistic Regression
0.702
0.6838785165589866
0.6838785165589866
nan
--------- planetoid_balanced 100 linqs_pubmed_planetoid 2 ---------


...

/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan
--------- planetoid_random_val_as_train 1000 linqs_pubmed_planetoid 8 ---------
========================= 60
randomize seed
random
========================= 560
Logistic Regression
0.732
0.7411912094795636
0.7411912094795636


/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan
--------- planetoid_random_val_as_train 1000 linqs_pubmed_planetoid 9 ---------
========================= 60
randomize seed
random
========================= 560
Logistic Regression
0.774
0.7819073967740252
0.7819073967740252
nan


/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [51]:
gcc_accs

array([[0.74693187, 0.74693187, 0.74693187, 0.74693187, 0.74693187,
        0.74693187, 0.74693187, 0.74693187, 0.74693187, 0.74693187,
        0.74693187],
       [0.68480392, 0.68480392, 0.68480392, 0.68480392, 0.68480392,
        0.68480392, 0.68480392, 0.68480392, 0.68480392, 0.68480392,
        0.68480392],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In [45]:
target_accs 
unlabeled_accs 
gcc_accs
nogcc_accs

array([[0.24878049, 0.24878049, 0.24878049, 0.24878049, 0.24878049,
        0.24878049, 0.24878049, 0.24878049, 0.24878049, 0.24878049,
        0.24878049],
       [0.16452442, 0.16452442, 0.16452442, 0.16452442, 0.16452442,
        0.16452442, 0.16452442, 0.16452442, 0.16452442, 0.16452442,
        0.16452442],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In [2]:
for i,data in enumerate(['linqs_cora_planetoid','linqs_citeseer_planetoid','linqs_pubmed_planetoid']):
    for j,seed in enumerate([-1,0,1,2,3,4,5,6,7,8,9]):
        true_labels, features, raw_features, graph, labeled_indices, unlabeled_indices, target_indices, gcc_indices, nogcc_indices = load_and_prepare_planetoid_data(data, seed=seed)
        num_nodes, num_classes = true_labels.shape
        labels, is_labeled = calc_masks(true_labels, labeled_indices,
                                        unlabeled_indices, raw_features)
        print(labeled_indices)

0
1
2
3
4
5
6
fixed seed
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139]
0
1
2
3
4
5
6
randomize seed
[  84   86   91   94   99  130  151  201  209  256  257  273  275  277
  307  324  368  373  388  423  430  508  538  545  556  561  608  638
  640  660  692  707  716  757  758  762  773  777  800  805  819  875
  891  934  957  975  976  979 1036 1074 1107 1155 1158 1170 1201 1207
 1209 1210 1211 1296 1308 1320 1333 1349 1356 1399 1438 1439 1470 1476
 1525 1541 1583 1594 1

KeyboardInterrupt: 